In [26]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from pickle import load

In [1]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [4]:
# load text
raw_text = load_doc('cleanedespn.txt')
print(raw_text[:200])

If the rumors are true and Doc Rivers gets a contract extension soon, I just hope the Celtics tell the truth in the ensuing news conference. Don't feed us lines like "Thanks to Doc, our young players 


In [5]:
# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)

In [8]:
# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
    # select sequence of tokens
    seq = raw_text[i-length:i+1]
    # store
    sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 5972194


In [9]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [12]:
# save sequences to file
out_filename = 'char_sequencesbs.txt'
save_doc(sequences, out_filename)

In [14]:
# load
in_filename = 'char_sequencesbs.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

In [15]:
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))

In [16]:
sequences = list()
for line in lines:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    sequences.append(encoded_seq)

In [17]:
# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 103


In [19]:
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

In [20]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

In [21]:
# define model
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 75)                53700     
_________________________________________________________________
dense_1 (Dense)              (None, 103)               7828      
Total params: 61,528
Trainable params: 61,528
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=5, verbose=2)

Epoch 1/5
 - 2364s - loss: 1.7688 - acc: 0.4993
Epoch 2/5
 - 2252s - loss: 1.5951 - acc: 0.5448
Epoch 3/5
 - 2385s - loss: 1.5620 - acc: 0.5535
Epoch 4/5
 - 2356s - loss: 1.5459 - acc: 0.5576
Epoch 5/5
 - 2390s - loss: 1.5356 - acc: 0.5602


In [23]:
# save the model to file
model.save('modelbs.h5')

In [24]:
# save the mapping
dump(mapping, open('mappingbs.pkl', 'wb'))

In [27]:
# load the model
model = load_model('modelbs.h5')

In [28]:
# load the mapping
mapping = load(open('mappingbs.pkl', 'rb'))

In [29]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

In [38]:
# test start of rhyme
print(generate_seq(model, mapping, 10, 'Doc Rivers ', 20))
# test mid-line
print(generate_seq(model, mapping, 10, 'king was i', 20))
# test not in original
print(generate_seq(model, mapping, 10, 'four and pi', 20))

Doc Rivers and the second that 
king was it to see the second 
four and pick to the second tha


In [39]:
#doesn't appear to be trained enough